# Generate mni subject

1. Imports
2. Download mni template
3. Apply reconall
4. Build surfaces from aseg

## 1. Import libraries

In [ ]:
from nipype import config
cfg = dict(logging=dict(workflow_level = 'DEBUG'),
           execution={'stop_on_first_crash': False,
                      'hash_method': 'content'})
config.update_config(cfg)

from nilearn.datasets import load_mni152_template,get_data_dirs, fetch_icbm152_2009,fetch_atlas_talairach
from nipype.interfaces.freesurfer.preprocess import ReconAll
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.utility import Function
import nipype.interfaces.freesurfer as fs
from nipype.interfaces.io import FreeSurferSource

import nibabel as nib
import numpy as np

import requests
import os

## 2. Download template and parcellations

In [ ]:
# Download mni template
data_dir = './mni-template/'
template = fetch_icbm152_2009(data_dir=data_dir)
print(template)

# Download brodmann areas from MRIcron github
url = 'https://github.com/neurolabusc/MRIcron/raw/master/niftiview7/templates/brodmann.nii.lut'
r = requests.get(url)
open(url.split('/')[-1],'w').write(os.path.join(data_dir,url.rsplit('/')[-1]))
url = 'https://github.com/neurolabusc/MRIcron/raw/master/niftiview7/templates/brodmann.nii.gz'
r = requests.get(url)
open(url.split('/')[-1],'w').write(os.path.join(data_dir,url.rsplit('/')[-1]))

# Download aal areas from MRIcron github
url = 'https://github.com/neurolabusc/MRIcron/raw/master/niftiview7/templates/aal.nii.gz'
r = requests.get(url)
open(url.split('/')[-1],'w').write(os.path.join(data_dir,url.rsplit('/')[-1]))
url = 'https://github.com/neurolabusc/MRIcron/raw/master/niftiview7/templates/aal.nii.lut'
r = requests.get(url)
open(url.split('/')[-1],'w').write(os.path.join(data_dir,url.rsplit('/')[-1]))

## 3. Apply FreeSurfer's recon-all to template

In [ ]:
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

reconall = ReconAll()
reconall.inputs.subject_id = 'mni'
reconall.inputs.directive = 'all'
reconall.inputs.subjects_dir = data_dir
reconall.inputs.T1_files = template['t1']
reconall.inputs.T2_file = template['t2']
reconall.inputs.use_T2 = True
reconall.inputs.openmp = 2
reconall.inputs.parallel = True
reconall.run()

### Generate surfaces from aseg

In [ ]:
NAMELIST = [
#[1,   'Left-Cerebral-Exterior',                  70,  130, 180, 0],
[2,   'Left-Cerebral-White-Matter',              245, 245, 245, 0],
[3,   'Left-Cerebral-Cortex',                    205, 62,  78,  0],
[4,   'Left-Lateral-Ventricle',                  120, 18,  134, 0],
[5,   'Left-Inf-Lat-Vent',                       196, 58,  250, 0],
#[6,   'Left-Cerebellum-Exterior',                0,   148, 0,   0],
[7,   'Left-Cerebellum-White-Matter',            220, 248, 164, 0],
[8,   'Left-Cerebellum-Cortex',                 230, 148, 34,  0],
[10,  'Left-Thalamus-Proper',                   0,   118, 14,  0],
[11,  'Left-Caudate',                            122, 186, 220, 0],
[12,  'Left-Putamen',                            236, 13,  176, 0],
[13,  'Left-Pallidum',                           12,  48,  255, 0],
[14,  '3rd-Ventricle',                           204, 182, 142, 0],
[15,  '4th-Ventricle',                           42,  204, 164, 0],
[16,  'Brain-Stem',                              119, 159, 176, 0],
[17,  'Left-Hippocampus',                        220, 216, 20,  0],
[18,  'Left-Amygdala',                           103, 255, 255, 0],
#[19,  'Left-Insula',                             80,  196, 98,  0],
[26,  'Left-Accumbens-area',                     255, 165, 0,   0],
[28,  'Left-VentralDC',                          165, 42,  42,  0],
#[40,  'Right-Cerebral-Exterior',                 70,  130, 180, 0],
[41,  'Right-Cerebral-White-Matter',             245, 245, 245, 0],
[42,  'Right-Cerebral-Cortex',                   205, 62,  78, 0],
[43,  'Right-Lateral-Ventricle',                 120, 18,  134, 0],
[44,  'Right-Inf-Lat-Vent',                      196, 58,  250, 0],
#[45,  'Right-Cerebellum-Exterior',               0,   148, 0,   0],
[46,  'Right-Cerebellum-White-Matter',           220, 248, 164, 0],
[47,  'Right-Cerebellum-Cortex',                 230, 148, 34,  0],
[49, 'Right-Thalamus-Proper',                  0,   118, 14,  0],
[50,  'Right-Caudate',                           122, 186, 220, 0],
[51,  'Right-Putamen',                           236, 13,  176, 0],
[52,  'Right-Pallidum',                          13,  48,  255, 0],
[53,  'Right-Hippocampus',                       220, 216, 20,  0],
[54,  'Right-Amygdala',                          103, 255, 255, 0],
#[55,  'Right-Insula',                            80,  196, 98,  0],
[58,  'Right-Accumbens-area',                    255, 165, 0,   0],
[60,  'Right-VentralDC',                         165, 42,  42,  0],
[251, 'CC_Posterior',                            0,   0,   64,  0],
[252, 'CC_Mid_Posterior',                        0,   0,   112, 0],
[253, 'CC_Central',                              0,   0,   160, 0],
[254, 'CC_Mid_Anterior',                         0,   0,   208, 0],
[255, 'CC_Anterior',                             0,   0,   255, 0]]

In [ ]:
lut_filepath = 'FreeSurferColorLUT.txt'
lut_array = np.genfromtxt(lut_filepath,
                          dtype=None,
                          usecols=(0, 1, 2, 3, 4, 5),
                          names=['id', 'name', 'R', 'G', 'B', 'A'],
                          encoding='utf-8')
bad_values = [0, 1, 6, 9, 19, 20, 21, 22, 23, 24, 25, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 45, 48, 55, 56, 57, 59, 61, 64, 65, 66, 67, \
              68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 96, 97, 98, 192,  1000]


# print(bad_values)

# Read aseg to extract the labels
fssource2 = FreeSurferSource(subjects_dir=data_dir,subject_id='mni')
res = fssource2.run()
q = nib.load(res.outputs.aparc_aseg[0])
volume_values = np.unique(q.get_data())
# print(volume_values)
# Delete all labels that are not in the volume
all_lut_idx = range(0,len(lut_array))

good_values_idx = []
for idx, elem in enumerate(lut_array):
    if elem[0] not in bad_values and elem[0] in volume_values and elem[0]<1000:
        good_values_idx.append(idx)

bad_values_idx = np.setdiff1d(all_lut_idx, good_values_idx)
lut_array = np.delete(lut_array, bad_values_idx)
print(lut_array)

In [ ]:
wf = Workflow(name="generate_surfaces_aseg")

fssource = Node(interface=FreeSurferSource(subjects_dir=os.path.abspath(data_dir),subject_id='mni'), name='fssource')
pretess = Node(interface=fs.MRIPretess(subjects_dir=os.path.abspath(data_dir)), name='pretess')
tess =  Node(interface=fs.MRITessellate(subjects_dir=os.path.abspath(data_dir)), name='tess')
smooth =  Node(interface=fs.SmoothTessellation(subjects_dir=os.path.abspath(data_dir)), name='smooth')
convert = Node(interface=fs.MRIsConvert(subjects_dir=os.path.abspath(data_dir),to_scanner=True), name='convert')

wf.connect(fssource, 'aseg', pretess, 'in_filled')
wf.connect(fssource, 'norm', pretess, 'in_norm')
wf.connect(pretess, 'out_file', tess, 'in_file')
wf.connect(tess, 'surface', smooth, 'in_file')
wf.connect(smooth, 'surface', convert, 'in_file')

In [ ]:
output_folder = os.path.join(os.path.abspath(data_dir), 'mni', 'nipype_surfaces')

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for l in lut_array:
    if not os.path.exists(os.path.join(output_folder, l[1]+'.surf.gii')):
        # Pre tessellation
        pretess.inputs.label = l[0]
        pretess.inputs.out_file = os.path.join(output_folder, l[1]+'_filled.mgz')
        pretess.outputs
        # Tesselation
        tess.inputs.label_value = l[0]
        tess.inputs.out_file = os.path.join(output_folder,l[1]+'_nonsmooth' )
        # Smoothing
        smooth.inputs.out_file = os.path.join(output_folder,l[1] )
        # Convert
        convert.inputs.out_file = os.path.join(output_folder, l[1]+'.surf.gii')
        # Run pipeline
        wf.run()      

        os.remove(pretess.inputs.out_file)
        os.remove(tess.inputs.out_file)
        os.remove(smooth.inputs.out_file)

In [ ]:
fssource = FreeSurferSource(subjects_dir=os.path.abspath(data_dir),subject_id='mni')
res = fssource.run()
print(output_folder)
pials = res.outputs.pial

left_pial_filename = ''
right_pial_filename = ''
for p in pials:
    if 'rh' in p:
        left_pial_filename = p
    elif 'lh' in p:
        right_pial_filename = p

# Convert rh.pial to Right-Cerebral-Cortex.surf.gii
convert = fs.MRIsConvert(subjects_dir=os.path.abspath(data_dir),to_scanner=True)
convert.inputs.in_file = left_pial_filename
convert.inputs.out_file = os.path.join(output_folder, 'Left-Cerebral-Cortex.surf.gii')
convert.run()    

# Convert rh.pial to Right-Cerebral-Cortex.surf.gii
convert = fs.MRIsConvert(subjects_dir=os.path.abspath(data_dir),to_scanner=True)
convert.inputs.in_file = right_pial_filename
convert.inputs.out_file = os.path.join(output_folder, 'Right-Cerebral-Cortex.surf.gii')
convert.run()     

### Generate surfaces from brodmann areas

In [ ]:
q = nib.load(os.path.join(os.path.abspath(data_dir),'brodmann.nii.gz'))
volume_values = np.unique(q.get_data())
print(volume_values)

In [ ]:
output_folder = os.path.join(os.path.abspath(data_dir), 'mni', 'brodmann_surfaces')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
        
for l in volume_values:
    label = 'Brodmann_'+str(l)
    if not os.path.exists(os.path.join(output_folder, label+'.vtk')):
        
        # Pre tessellation
        pretess = fs.MRIPretess(subjects_dir=os.path.abspath(data_dir))
        pretess.inputs.label = l
        pretess.inputs.in_filled = os.path.join(data_dir,'brodmann.nii.gz')
        pretess.inputs.in_norm = os.path.join(data_dir,'brodmann.nii.gz')
        pretess.inputs.out_file = os.path.join(output_folder, label+'_filled.mgz')
        pretess.run()
        # Tesselation
        tess =  fs.MRITessellate(subjects_dir=os.path.abspath(data_dir))
        tess.inputs.label_value = l
        tess.inputs.in_file = pretess.inputs.out_file
        tess.inputs.out_file = os.path.join(output_folder,label+'_nonsmooth' )
        tess.run()
        # Smoothing
        smooth =  fs.SmoothTessellation(subjects_dir=os.path.abspath(data_dir))
        smooth.inputs.in_file = tess.inputs.out_file
        smooth.inputs.out_file = os.path.join(output_folder,label )
        smooth.run()
        # Convert
        convert = fs.MRIsConvert(subjects_dir=os.path.abspath(data_dir),to_scanner=True)
        convert.inputs.in_file = smooth.inputs.out_file
        convert.inputs.out_file = os.path.join(output_folder, label+'.vtk')
        convert.run()      

        os.remove(pretess.inputs.out_file)
        os.remove(tess.inputs.out_file)
        os.remove(smooth.inputs.out_file)

### Generte surfaces from brodmann areas

In [ ]:
q = nib.load(os.path.join(os.path.abspath(data_dir),'aal.nii.gz'))
volume_values = np.unique(q.get_data())
print(volume_values)

In [ ]:
output_folder = os.path.join(os.path.abspath(data_dir), 'mni', 'aal_surfaces')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
        
for l in volume_values:
    label = 'AAL_'+str(l)
    if not os.path.exists(os.path.join(output_folder, label+'.vtk')):
        
        # Pre tessellation
        pretess = fs.MRIPretess(subjects_dir=os.path.abspath(data_dir))
        pretess.inputs.label = l
        pretess.inputs.in_filled = os.path.join(data_dir,'aal.nii.gz')
        pretess.inputs.in_norm = os.path.join(data_dir,'aal.nii.gz')
        pretess.inputs.out_file = os.path.join(output_folder, label+'_filled.mgz')
        pretess.run()
        # Tesselation
        tess =  fs.MRITessellate(subjects_dir=os.path.abspath(data_dir))
        tess.inputs.label_value = l
        tess.inputs.in_file = pretess.inputs.out_file
        tess.inputs.out_file = os.path.join(output_folder,label+'_nonsmooth' )
        tess.run()
        # Smoothing
        smooth =  fs.SmoothTessellation(subjects_dir=os.path.abspath(data_dir))
        smooth.inputs.in_file = tess.inputs.out_file
        smooth.inputs.out_file = os.path.join(output_folder,label )
        smooth.run()
        # Convert
        convert = fs.MRIsConvert(subjects_dir=os.path.abspath(data_dir),to_scanner=True)
        convert.inputs.in_file = smooth.inputs.out_file
        convert.inputs.out_file = os.path.join(output_folder, label+'.vtk')
        convert.run()      

        os.remove(pretess.inputs.out_file)
        os.remove(tess.inputs.out_file)
        os.remove(smooth.inputs.out_file)